# Develop Model Driver

In this notebook, we will develop the API that will call our model. This module initializes the model, transforms the input so that it is in the appropriate format and defines the scoring method that will produce the predictions. The API will expect the input to be in JSON format. Once a request is received, the API will use the request body to score the question text. There are two main functions in the API. The first function loads the model and returns a scoring function. The second function process the question text and uses the first function to score it.

In [1]:
import pandas as pd
import logging
import json

We use the writefile magic to write the contents of the below cell to driver.py which includes the driver methods.

In [2]:
%%writefile driver.py

import lightgbm as lgb
import timeit as t
import logging
from duplicate_model import DuplicateModel

model_path = 'model.pkl'
questions_path = 'questions.tsv'
logger = logging.getLogger("model_driver")

def create_scoring_func():
    """ Initialize Model Object 
    """   
    start = t.default_timer()
    DM = DuplicateModel(model_path, questions_path)
    end = t.default_timer()
    
    loadTimeMsg = "Model object loading time: {0} ms".format(round((end-start)*1000, 2))
    logger.info(loadTimeMsg)
    
    def call_model(text):
        preds = DM.score(text)  
        return preds
    
    return call_model

def get_model_api():
    logger = logging.getLogger("model_driver")
    scoring_func = create_scoring_func()
    
    def process_and_score(inputString):
        """ Classify the input using the loaded model
        """
        start = t.default_timer()

        responses = []
        preds = scoring_func(inputString)
        responses.append(preds)

        end = t.default_timer()
        
        logger.info("Predictions: {0}".format(responses))
        logger.info("Predictions took {0} ms".format(round((end-start)*1000, 2)))
        return (responses, "Computed in {0} ms".format(round((end-start)*1000, 2)))
    return process_and_score

def version():
    return lgb.__version__

Writing driver.py


Let's test the module.

In [3]:
logging.basicConfig(level=logging.DEBUG)

We run the driver.py which will bring the imports and functions into the context of the notebook.

In [4]:
%run driver.py

Now, let's use one of the duplicate questions to test our driver.

In [5]:
dupes_test_path = 'dupes_test.tsv'
dupes_test = pd.read_csv(dupes_test_path, sep='\t', encoding='latin1')
text_to_score = dupes_test.iloc[0,4]
text_to_score

"get total number of items on json object?.  possible duplicate: length of javascript object (ie. associative array)   i have an object similar to this one:  i'm trying to get it's length, the problem is that jsonarray.length returns 5 instead of 3 (which is the total items it has). the array is relatively long (has 1000x2000 items) and this must be done a lot of times every second. how can i get the number of items more efficiently?"

Here, we define a helper function to convert our text for the format that will be required by the Flask application that will use the functions in the driver.

In [6]:
def text_to_json(text):
    return json.dumps({'input':'{0}'.format(text)})

In [7]:
jsontext = text_to_json(text_to_score)
json_load_text = json.loads(jsontext)
body = json_load_text['input']

In [8]:
predict_for = get_model_api()

INFO:model_driver:Model object loading time: 276.32 ms


In [9]:
resp = predict_for(body)

INFO:model_driver:Predictions: [[(126100, 4889658, 0.9296362731106259), (11922383, 11922384, 0.8044288817585545), (5223, 6700, 0.7198366172405716), (122102, 122704, 0.2890961841298208), (8495687, 8495740, 0.11294396697164998), (14220321, 14220323, 0.09135746055182127), (750486, 750506, 0.06237155946634724), (12953704, 12953750, 0.05119111679990697), (19590865, 19590901, 0.05030352671033994), (2274242, 2274327, 0.03262457462662157), (4255472, 4255480, 0.03261161556294271), (237104, 1473742, 0.02543085185321038), (149055, 149150, 0.023086864548377065), (7486085, 7486130, 0.022343142672530538), (20279484, 20279485, 0.021296540519782723), (4616202, 4616273, 0.02056208401842061), (1069666, 1069840, 0.017562602019344658), (23667086, 23667087, 0.015999224180601113), (171251, 171256, 0.01440360115394622), (4935632, 4935684, 0.014354337793775984), (1129216, 1129270, 0.01406801949758789), (5767325, 5767357, 0.013094310211096949), (307179, 307200, 0.01254909445661112), (684672, 684692, 0.01063624

Next, we move on to building our docker image.